## IMPORT LIBRARY

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import sqlite3

import warnings
warnings.filterwarnings('ignore')

## LOGGING

In [2]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)

    with open('code_log.txt', 'a') as f:
        f.write(timestamp + ' : ' + message + '\n')

## EXTRACT

In [3]:
url = 'https://companiesmarketcap.com/banks/largest-banks-by-market-cap/'
table_attribs = ['Name', 'MC_USD_Billion']

def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    td_elements = soup.find_all('td', class_='name-td')

    for td in td_elements:
        bank_name = td.find('div', class_='company-name').text.strip()
        market_cap = td.find_next('td', class_='td-right').text.strip()
        market_cap = market_cap.replace('$', '').replace('B', '')
        market_cap_numeric = float(market_cap) if market_cap else None

        data_dict = {'Name': bank_name, 'MC_USD_Billion': market_cap_numeric}
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df, df1], ignore_index=True)

    log_progress('Data extraction complete. Initiating Transformation process')
    return df

In [4]:
df = extract(url, table_attribs)
df

,Name,MC_USD_Billion
0,JPMorgan Chase,503.50
1,Bank of America,260.66
2,ICBC,238.75
3,Agricultural Bank of China,187.80
4,Wells Fargo,173.03
...,...,...
95,The Saudi British Bank,20.72
96,Cathay Financial Holding,20.47
97,Hang Seng Bank,19.80
98,KB Financial Group,19.50


## TRANSFORM

In [5]:
csv_path = 'exchange_rate.csv'

def transform (df, csv_path):
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']

    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    log_progress('Data transformation complete. Initiating Loading process')
    return df

In [6]:
df = transform(df, csv_path)
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,503.50,402.80,468.26,41765.33
1,Bank of America,260.66,208.53,242.41,21621.75
2,ICBC,238.75,191.00,222.04,19804.31
3,Agricultural Bank of China,187.80,150.24,174.65,15578.01
4,Wells Fargo,173.03,138.42,160.92,14352.84
...,...,...,...,...,...
95,The Saudi British Bank,20.72,16.58,19.27,1718.72
96,Cathay Financial Holding,20.47,16.38,19.04,1697.99
97,Hang Seng Bank,19.80,15.84,18.41,1642.41
98,KB Financial Group,19.50,15.60,18.14,1617.52


## LOAD

In [7]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

    log_progress('Data saved to CSV file')

csv_output_path = 'Largest_banks_data.csv'
load_to_csv(df, csv_output_path)

In [8]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Data loaded to Database as a table, Executing queries')

database_name = 'Banks.db'
table_name = 'Largest_banks'

connection = sqlite3.connect(database_name)
load_to_db(df, connection, table_name)
connection.close()

## RUNNING QUERIES

In [9]:
connection = sqlite3.connect('Banks.db')

In [10]:
query1 = 'SELECT * FROM Largest_banks'
result1 = pd.read_sql_query(query1, connection)
result1

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,503.50,402.80,468.26,41765.33
1,Bank of America,260.66,208.53,242.41,21621.75
2,ICBC,238.75,191.00,222.04,19804.31
3,Agricultural Bank of China,187.80,150.24,174.65,15578.01
4,Wells Fargo,173.03,138.42,160.92,14352.84
...,...,...,...,...,...
95,The Saudi British Bank,20.72,16.58,19.27,1718.72
96,Cathay Financial Holding,20.47,16.38,19.04,1697.99
97,Hang Seng Bank,19.80,15.84,18.41,1642.41
98,KB Financial Group,19.50,15.60,18.14,1617.52


In [11]:
query2 = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
result2 = pd.read_sql_query(query2, connection)
result2

,AVG(MC_GBP_Billion)
0,51.999


In [12]:
query3 = 'SELECT Name FROM Largest_banks LIMIT 5'
result3 = pd.read_sql_query(query3, connection)
result3

,Name
0,JPMorgan Chase
1,Bank of America
2,ICBC
3,Agricultural Bank of China
4,Wells Fargo


In [13]:
connection.close()